In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import pandas as pd
import cv2

import tensorflow as tf
from tensorflow import keras

import PIL
from PIL import Image 
from PIL.ImageDraw import Draw

In [ ]:
batch_size = 32

In [ ]:
def load_images(img_names, model_size):
    """Loads images in a 4D array.

    Args:
        img_names: A list of images names.
        model_size: The input size of the model.
        data_format: A format for the array returned
            ('channels_first' or 'channels_last').

    Returns:
        A 4D NumPy array.
    """
    imgs = []

    for img_name in img_names:
        img = Image.open(img_name)
        img = img.resize(size=model_size)
        img = np.array(img, dtype=np.float32)
        img = np.expand_dims(img, axis=0)
        imgs.append(img)

    imgs = np.concatenate(imgs)

    return imgs

In [8]:
cam = cv2.VideoCapture(1) 
cam.set(cv2.CAP_PROP_FPS, 120)

while True:
    ret, image = cam.read()
    cv2.imshow('Camera Feed', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


1   HIToolbox                           0x000000018ecf45c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x000000018ecbe478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018a1d1214 _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018a1cfe10 CGSSnarfAndDispatchDatagrams + 808
5   SkyLight                            0x000000018a4f7a9c SLSGetNextEventRecordInternal + 344
6   SkyLight                            0x000000018a2f8fb0 SLEventCreateNextEvent + 16
7   HIToolbox                           0x000000018ec8cb58 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 60
8   HIToolbox                           0x000000018ec8cae0 _ZL14MessageHandlerP12__CFMachPortPvlS1_ + 60
9   CoreFoundation                      0x00000001855d1410 __CFMachPortPerform + 260
10  CoreFoundation                      0x00000001855a1f98 __CFRUNL

KeyboardInterrupt: 

: 